In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [ ]:
%cd '/content/drive/MyDrive/EEG project'
data = np.load(file="rgb_eeg.npy")
labels = np.load(file='labels.npy')
from sklearn.utils import shuffle
data,labels = shuffle(data, labels, random_state=0)
train_X= data[:4900]
train_y= labels[:4900]
test_X= data[4900:]
test_y= labels[4900:]

/content/drive/MyDrive/EEG project


In [ ]:
len(data)

5406

In [ ]:
print(test_X.shape)
print(test_y.shape)

(506, 20, 20, 3)
(506,)


In [ ]:
import torch
import pandas as pd
from torch import nn,optim
import torchvision
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms, models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable


In [ ]:
class MIDataset(torch.utils.data.Dataset):
	#dataset 
    def __init__(self, input_data, labels, transform=None):
      self.input_data = input_data
      self.labels = labels
      self.transform = transform
 
    def __len__(self):
      return len(self.labels)
 
    def __getitem__(self, index):
      data = self.input_data[index]
      label = self.labels[index]
      if self.transform is not None:
        data = self.transform(data)
      return data,label

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.228, 0.224, 0.225]
transform_train = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

transform_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [ ]:
train_data = MIDataset(train_X,train_y,transform_train)
test_data = MIDataset(test_X,test_y,transform_test)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_data,batch_size=64,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data,batch_size=64,shuffle=False)

In [ ]:
lr = 1e-4
BATCH_SIZE = 64
EPOCHS = 10
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# 实例化模型并且移动到GPU
model = models.resnet18(pretrained=True)
numFit = model.fc.in_features
numClass = 3
model.fc = nn.Sequential(nn.Linear(numFit, numClass), nn.Softmax(dim=1))
criterion = nn.CrossEntropyLoss()
model.to(DEVICE)
# 选择Adam优化器
optimizer = optim.Adam(model.parameters(), lr=lr)

def adjust_lr(optimizer, epoch):
    new_lr = lr * (0.1 ** (epoch // 20))
    print("lr:", new_lr)
    for param_group in optimizer.param_groups:
        param_group['lr'] = new_lr


In [ ]:
# 定义训练过程

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    total_loss = 0
    total_num = len(train_loader.dataset)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).to(device), Variable(target).to(device)
        target = target.to(dtype=torch.long)
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print_loss = loss.data.item()
        total_loss += print_loss

    avg_loss = total_loss / len(train_loader)
    print('epoch:{},loss:{}'.format(epoch, avg_loss))
    return avg_loss

def val(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total_num = len(test_loader.dataset)
    print(total_num, len(test_loader))
    with torch.no_grad():
        for data, target in test_loader:
            data, target = Variable(data).to(device), Variable(target).to(device)
            target = target.to(dtype=torch.long)
            output = model(data)
            loss = criterion(output, target)
            _, predict = torch.max(output.data, 1)
            correct += torch.sum(predict == target)
            loss = loss.data.item()
            total_loss += loss
        correct = correct.data.item()
        acc = correct / total_num
        avgloss = total_loss / len(test_loader)
        print('\nValidation: avg loss: {:.4f}, acc: {}/{} ({:.0f}%)\n'.format(
            avgloss, correct, len(test_loader.dataset), 100 * acc))
        return (avgloss,acc)



train_l = []
test_l = []
for epoch in range(1, EPOCHS + 1):
    adjust_lr(optimizer, epoch)
    train_loss = train(model, DEVICE, train_loader, optimizer, epoch)
    test_loss, test_acc = val(model, DEVICE, test_loader)
    train_l.append(train_loss)
    test_l.append(test_loss)

In [ ]:

save_dict = {"state_dict": model.state_dict()}
torch.save(
    save_dict,
    os.path.join('./baseline_train', "checkpoint.pth.tar"),
    _use_new_zipfile_serialization=False,
)